In [494]:
import numpy as np
import pandas as pd
from math import factorial as fact

def get_inverse(state, size, loc, solvable=False): # 단, Hash 구하는 용도임(solvable xx)
    # input 
    #   state : 퍼즐 (2d array)
    #   size : 퍼즐 사이즈 (tuple)
    #   loc : inverse 값을 구하려는 타일 좌표 (tuple)(row, col)
    # output
    #   inverse value : 우하단 방향으로, 해당 좌표의 값보다 작은 타일의 개수.

    m, n = size
    y, x = loc
    anchor_value = state[y, x] # 타일의 값
    count = 0
    
    for i in range(y, m):
        if (i==y):# 첫 줄
            for j in range(x+1, n):
                if(state[i, j] < anchor_value):
                    if solvable:
                        if state[i,j]!=0: # solvable 구할 때는 blanktile 포함노
                            count+=1
                    else:
                        count +=1

        else: # 나머지 줄 
            for j in range(0, n):
                if(state[i, j] < anchor_value):
                    if solvable:
                        if state[i,j]!=0:
                            count+=1
                    else:
                        count+=1

    return count

# 3x2 Sub-puzzle에서, 6!/2 = 360가지의 puzzle -> 360개의 hash 
def hash(state):

    hash_value = 1
    i = 0
    m, n = np.shape(state)
    for row in range(m-1, -1, -1):
        for col in range(n-1, -1, -1):
            temp_inverse = get_inverse(state, (m, n), (row, col))

            hash_value += (temp_inverse * fact(i))
            i+=1 # 좌표 
    
    return hash_value

# 대충 만든 swap 함수 (어차피 in-place Algorithm이라 굳이 다른 자료구조 안 써도 괜찮)
def move(state, size, blank_loc, direct):
    m, n = size
    y, x = blank_loc
    if direct=="left":
        if x-1<0:
            return
        else:
            state[y, x], state[y, x-1] = state[y, x-1], state[y, x] # swap 
            return (y, x-1)
    elif direct=="right":
        if x+1>=n:
            return
        else:
            state[y, x], state[y, x+1] = state[y, x+1], state[y, x] # swap
            return (y, x+1)

    elif direct=="up":
        if y-1<0:
            return
        else:   
            state[y, x], state[y-1, x] = state[y-1, x], state[y, x] # swap
            return (y-1, x)
    elif direct=="down":
        if y+1>=m:
            return
        else:
            state[y, x], state[y+1, x] = state[y+1, x], state[y, x] # swap
            return (y+1, x)

    else:
        raise NameError("제대로 옮기세요")   


# SST table 만들기용 move 함수 (C++에선 Struct)
def move_sst(state2, size, blank_loc, direct):
    state = state2.copy() # deepcopy

    m, n = size
    y, x = blank_loc
    if direct=="left":
        if x-1<0:
            return
        else:
            state[y, x], state[y, x-1] = state[y, x-1], state[y, x] # swap 
            return state, (y, x-1)
    elif direct=="right":
        if x+1>=n:
            return
        else:
            state[y, x], state[y, x+1] = state[y, x+1], state[y, x] # swap
            return state, (y, x+1)

    elif direct=="up":
        if y-1<0:
            return
        else:   
            state[y, x], state[y-1, x] = state[y-1, x], state[y, x] # swap
            return state, (y-1, x)
    elif direct=="down":
        if y+1>=m:
            return
        else:
            state[y, x], state[y+1, x] = state[y+1, x], state[y, x] # swap
            return state, (y+1, x)

    else:
        raise NameError("제대로 옮기세요") 





In [601]:
# Bidirectional BFS  / State Transition Table을 만드는 용도.
def BBFS(target_state, size):
    hash_array=np.zeros(1000) # hash_array[hash_value] = state sequence
    order = {0: "up", 1:"right", 2:"down", 3:"left"}
    m,n = size # (3,2) 혹은 (2,3)

    hash_value = hash(target_state)
  

    next_seq=1
    prev_seq = -1
    reverse_direct = "out"
    hash_array[hash_value]=next_seq # state 방문 처리
    
    queue=[]

    # 현재 state, blank 위치, sequence,  hash value, previous sequence )
    queue.append((target_state, (m-1, n-1), next_seq, hash_value, prev_seq, reverse_direct))
    # c에선 구조체로 바꿔주자.


    full_seq=[]
    full_hash=[]
    full_prev_seq=[]
    full_reverse_direct=[]
    while queue:

        # print("queue: [", end=' ')
        # for i in queue:
        #     print(i[2], end =' ')
        # print("]")

        current = queue.pop(0)

        # print("---pop ! --- :")
        # print("state: \n", current[0], "sequence : ",  current[2], " hash value : ", current[3], " previous sequence : ", current[4], " previous recverse direct : ", current[5])
        # print("----------------")

        prev_state = current[0] # 일단 구조체가 아니라 풀어서 씀.
        prev_blank = current[1]
        prev_seq = current[2]
        prev_hash = current[3]  
        prev_prev_seq = current[4]
        prev_reverse_direc = current[5]

        full_seq.append(prev_seq)
        full_hash.append(prev_hash)
        full_prev_seq.append(prev_prev_seq)
        full_reverse_direct.append(prev_reverse_direc)
        
        
        for key, direct in order.items():
            reverse_direct = order[(key+2)%4] # 백트래킹용 역방향
            next_state_and_loc  = move_sst(prev_state, size, prev_blank, direct)

            if next_state_and_loc: # 움직일 수 있으면
                next_hash = hash(next_state_and_loc[0]) # hash값 추출
                if not hash_array[next_hash]: # 아직 방문되지 않았으면, 
                    
                    next_seq = next_seq + 1
                    queue.append((next_state_and_loc[0], next_state_and_loc[1], next_seq, next_hash, prev_seq, reverse_direct)) # 큐에 삽입
                    
                    

                    # print("---insert !---" )

                    # print("queue: [", end=' ')
                    # for i in queue:
                    #     print(i[2], end =' ')
                    # print("]")

                    
                    # print("state: ", next_state_and_loc[0], "sequence : ",  next_seq, " hash value : " , next_hash, " previous sequence : ", prev_seq)
                    # print("-------------------------------")
                    hash_array[next_hash] = next_seq # 방문처리 
                else:
                    pass
                    # print("이미 방문한 해쉬입니다. : ", next_hash)
    

    STT=pd.DataFrame({"seq":full_seq, "hash":full_hash, "prev_seq":full_prev_seq, "reverse_direct": full_reverse_direct})
    return STT, hash_array


# STT를 이용해 3x2 Sub-puzzle을 최단경로로 풀어주는 함수      
def GetShortestPath(STT, hash_array, initial_state):
    
    hash_value = hash(initial_state)
    
    
    next_seq=int(hash_array[hash_value]) # initial state
    prev_seq = STT.iloc[next_seq-1]["prev_seq"] # next seq -> prev seq ()

    full_direct=[]
    while (prev_seq!=-1): # 각 행의 마지막 2개 타일 정렬이 끝날 때까지
        next_direct = STT.iloc[next_seq-1]["reverse_direct"] # 더 가야 한다.
        # print(next_seq, next_direct)
        full_direct.append(next_direct) # 방향 저장
        
        next_seq = prev_seq
        prev_seq = STT.iloc[next_seq-1]["prev_seq"]


    
    # 최적화 더 해야함. 굳이 끝까지 맞출 필요 없음.(함수 더 짤 때 ㄱㄱ)
        
 
    
    return full_direct

# 3x2 puzzle을 풀기위해 [1,2,3,4,5,0] 형태로 정규화가 필요함.
def MatchToStandard(initial_state, t1, t2, last=False): # last : Last 2x3 Puzzle
    # [4, 8, 5, 11, 0, 15] -> [1, *, 2, *, 0, ]  ( ***는 3,4,5 중 하나)
    # t1 : 4 / t2 : 5 (in row 1)
    # 단, ***를 345로 배열하면 inverse : 1+2+2+2+1 = 짝수. 

    m, n = np.shape(initial_state)    

    standard_initial_state = np.zeros((m, n))

    if (m,n)==(3,2):
        if initial_state[2, 0]==0: # 
            others = [5,4,3] # for solvable
            
        else:
            others = [3,4,5] # for solvable
        
    else: # 아마 2x3 퍼즐만.
        if not last:
            others = [2,3,5] # Solvable
        else: # last
            pass

        
    idx=0        
    for i in range(m):
        for j in range(n):
            if (initial_state[i,j] == t1):
                standard_initial_state[i,j] = 1
            elif (initial_state[i,j]==t2):
                if (m,n)==(3,2):
                    standard_initial_state[i,j] = 2
                else: 
                    standard_initial_state[i,j] = 4
                    
            elif (initial_state[i,j]==0):
                standard_initial_state[i,j] = 0
            else: # 나머진 대충
                if not last:
                    standard_initial_state[i,j] = others[idx]
                    idx+=1
                if last:
                    if (initial_state[i,j]==23):
                        standard_initial_state[i,j] = 2
                    elif (initial_state[i,j]==24):
                        standard_initial_state[i,j] = 3
                    else: # 29
                        standard_initial_state[i,j] = 5

    return standard_initial_state
    

    

    

# Solving Idea : 한 줄 한 줄씩 맞추기
### - Stage 1 : 각 행의 5개 열은 그냥 Shortest path  
### - Stage 2 : 각 행의 마지막 2개 타일은 hash + State Transition Table(STT) 활용한 3x2 Sub-puzzle 
### - Stage 3 : 마지막 2행은 총 4개의 2x3 Sub-puzzle을 풀어야함

 -------------

#### 0. Initialize : Bidirectional Breadth First Search를 통한 hash + STT 생성 (0.1초)

In [620]:
target_state = np.array([[1,2], [3,4], [5,0]])
target_state2 = np.array([[1,2,3], [4,5,0]])

STT,hash_array = BBFS(target_state, size=(3,2))
STT2, hash_array2= BBFS(target_state2, size=(2,3))

#### 1.1 Row 1 (전방 4개) : 최단경로로 맞출 수 있음 
#### 1.2 Row 1 (후방 2개) : STT 활용 최단경로

In [621]:
initial_state = np.array([[5, 24], [6, 15], [0, 22]]) # 3x2 sub-puzzle
initial_state = MatchToStandard(initial_state, 5, 6) # 정규화
# 13 : 5, 9 : 4, 21 : 3
# 이거 Target state를 그냥 [[1,2][4,0][3, 5]] 정도로 하면 이득일듯.
# 즉, [[t1, *], [t2, *], [0, *]] -> [[1,2], [4,0], [3,5]]
# 0이 오른쪽에 있을 때는..? (사실 그럴일이 별로 없긴 함) - 케이스 분석해야
print(GetShortestPath(STT, hash_array,initial_state))

['up', 'right', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'up', 'right', 'down', 'left', 'down', 'right']


#### 2.1 Row 2 (전방 4개) : 최단경로로 맞출 수 있음 
#### 2.2 Row 2 (후방 2개) : STT 활용 최단경로

In [622]:
initial_state = np.array([[11,25], [12, 15], [0, 16]]) # 3x2 sub-puzzle
initial_state = MatchToStandard(initial_state, 11, 12) # 정규화a

print(GetShortestPath(STT, hash_array,initial_state, target_state))

['up', 'right', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'up', 'right', 'down', 'left', 'down', 'right']


#### 3.1 Row 3(전방 5개) : 최단경로
#### 3.2 Row 3(후방 2개) : STT 활용 최단경로

In [623]:
initial_state = np.array([[17,29], [18, 25], [0, 24]]) # 3x2 sub-puzzle
initial_state = MatchToStandard(initial_state, 17, 18) # 정규화a

print(GetShortestPath(STT, hash_array,initial_state, target_state))

['up', 'right', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'up', 'right', 'down', 'left', 'down', 'right']


#### 4.1. 마지막 Row 2개(전방 3개) : 2x3 Sub Puzzle STT 활용 최단 경로

In [626]:
initial_state2 = np.array([[21, 27, 0], [28, 22, 24]]) # 3x2 sub-puzzle
initial_state2 = MatchToStandard(initial_state2, 21, 27) # 정규화

print(GetShortestPath(STT2, hash_array2,initial_state2))

['left', 'down', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'down', 'right', 'right', 'up', 'left', 'down', 'right']


#### 4.2. 마지막 2x3 Sub Puzzle : STT 활용 최단 경로

In [627]:
initial_state2 = np.array([[0, 24, 29], [22,28, 23]]) # 3x2 sub-puzzle
initial_state2 = MatchToStandard(initial_state2, 22, 28,
                 last=True) # 정규화

print(GetShortestPath(STT2, hash_array2,initial_state2))

['down', 'right', 'right', 'up', 'left', 'down', 'right']


## 408번(시간은 0.x초 예상)